In [ ]:
import torch
from torchdiffeq import odeint
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.ticker import MaxNLocator
import seaborn as sns
import pickle
import pandas as pd

from abc import ABC, abstractmethod
import time, random, itertools
from typing import Callable, Tuple, List, Optional
from scipy.integrate import solve_ivp
from scipy.interpolate import RegularGridInterpolator
import tqdm
import os

from scripts.utils import set_seed
from scripts.ds_class import *
from scripts.homeos import *
from scripts.plotting import *
from scripts.fit_motif import *
from scripts.time_series import *
from scripts.ra import *
from scripts.exp_tools import *
set_seed(313)
exp_folder = 'experiments/'

In [ ]:
ds_motif_list = ['ring'] #, 'lc', 'lds', 'bla']
noise_std_list = [0.0]  
random_seed_list = [0, 10, 20, 100, 200]
for noise_std in noise_std_list:
    for ds_motif in ds_motif_list:
        for random_seed_run in random_seed_list:
            perthomeo_exp(ds_motif=ds_motif, noise_std=noise_std, random_seed_run=random_seed_run)

In [ ]:
# #load data
# save_dir_homeo = "experiments/homeopert_ring/ring"
# df = pd.read_pickle(f"{save_dir_homeo}/summary_df.pkl")

In [1]:
# ###test models
# inv_man_list = []
# jac_norm_list = []
# for i, row in df.iterrows():
#     interpol_value = row['interpol_value']
#     all_parameters = pickle.load(open(save_dir_homeo+"/parameters.pkl", "rb"))
#     all_parameters['homeo_params']['layer_sizes'] = [128]
#     all_parameters['simulation_params']['ds_params']['alpha_init'] = None
#     all_parameters['simulation_params']['ds_params']['canonical'] = True
#     homeo = build_homeomorphism(all_parameters['homeo_params'])
#     dynsys = build_ds_motif(**all_parameters['simulation_params']['ds_params'])
#     #dynsys.alpha = torch.nn.Parameter(torch.tensor(-1.))
#     homeo_ds_net = load_homeo_ds_net(f"{save_dir_homeo}/homeo_{interpol_value}.pth", homeo, dynsys)
#     homeo_ds_net.eval()
#     homeo_ds_net.dynamical_system.alpha = -1.
#     trajectories_target = np.load(f"experiments/homeopert_ring/trajectories_target_{row.interpol_value}.npy")
#     trajectories_target = torch.tensor(trajectories_target, dtype=torch.float32)
#     traj_src_np, traj_trans_np, _ = test_single_homeo_ds_net(homeo_ds_net=homeo_ds_net, trajectories_target=trajectories_target)
#     traj_trans = torch.tensor(traj_trans_np, dtype=torch.float32)
#     inv_man = homeo_ds_net.invariant_manifold()
#     inv_man_list.append(inv_man)

#     # jac_norm = jacobian_norm_over_batch(homeo_ds_net.homeo_network, traj_trans.reshape(-1, all_parameters['simulation_params']['ds_params']['dim']), norm_type='spectral')
#     # #jac_norm_list.append(jac_norm)
#     # jac_norm2 = jacobian_spectral_norm(homeo_ds_net.homeo_network, traj_trans, subtract_identity=True, normalize=True, n_iter=20)
#     # print(f"Interpol value: {interpol_value}, Jacobian norm: {jac_norm.item()}", jac_norm2.item())

#     # np.save(f"{save_dir_homeo}/traj_motif_transformed_{interpol_value}.npy", traj_trans_np) 
#     # np.save(f"{save_dir_homeo}/traj_motif_source_{interpol_value}.npy", traj_src_np) #save trajectories_motif directly from the source 

In [2]:
# target_jac_norm_list = []
# dim = all_parameters['simulation_params']['ds_params']['dim']
# dt = all_parameters['simulation_params']['dt']
# time_span = all_parameters['simulation_params']['time_span']
# generator_ra = AnalyticalRingAttractor(dim=dim, dt=dt, time_span=time_span)
# simulation_params = all_parameters['simulation_params']
# init_conds = prepare_initial_conditions(
#     mode=simulation_params['initial_conditions_mode'],
#     num_points=simulation_params['number_of_target_trajectories'],
#     margin=simulation_params['margin'],
#     seed=simulation_params['seed']
# )
# ra_trajs = generator_ra.compute_trajectory(torch.tensor(init_conds, dtype=torch.float32))
# for i, row in df.iterrows():
#     interpol_value = row['interpol_value']
#     target_homeo = build_homeomorphism(all_parameters['target_homeo_params'])
#     interpolated_homeo = rescale_node_vf(target_homeo, interpol_value)
#     jac_norm = jacobian_norm_over_batch(interpolated_homeo, ra_trajs.reshape(-1, dim), norm_type='spectral')
#     target_jac_norm_list.append(jac_norm)

In [9]:
#time length generalization
# length_factor = 10
# maxT_long = maxT/2*length_factor
# tsteps_long = (tsteps-15)*length_factor
# time_span_long = torch.tensor([0.0, time_span[1]*length_factor])
# init_conds_trg = torch.tensor(inv_man, dtype=torch.float32)
# trajectories_source_long, transformed_trajectories_long = generate_trajectories_from_initial_conditions(homeo_ds_net, init_conds_trg, time_span_long)

## Vector field perturbation experiment

In [ ]:
ds_motif_list = ['ring'] #['ring', 'lc', 'lds', 'bla', 'bistable']
for ds_motif in ds_motif_list:
    for random_seed_run in random_seed_list:
        run_pert_ra_experiment(ds_motif=ds_motif)

In [ ]:
# #load data
# save_dir_pertvf_sra = "experiments/vf_pert_ring/simple"
# df = pd.read_pickle(f"{save_dir_pertvf_sra}/summary_df.pkl")

In [14]:
# test models 
# for i, row in df.iterrows():
#     p_norm = row['p_norm']
#     all_parameters = pickle.load(open(f"{save_dir_pertvf_sra}/parameters.pkl", "rb"))
#     layer_sizes = all_parameters['homeo_params']['layer_sizes']
#     alpha_init = all_parameters['simulation_params']['ds_params']['alpha_init']
#     homeo = NODEHomeomorphism(dim=2, layer_sizes=[128], activation=nn.ReLU)        
#     dynsys = AnalyticalRingAttractor(dim=2, alpha_init=None, dt=0.05, time_span=torch.tensor([0.0, 5.]))
#     homeo_ds_net = load_homeo_ds_net(f"{save_dir_pertvf_sra}/homeo_{p_norm}.pth", homeo, dynsys)
#     homeo_ds_net.eval()
#     trajectories_target = np.load(f"{save_dir_pertvf_sra}/trajectories_target_{row.p_norm}.npy")
#     trajectories_target = torch.tensor(trajectories_target, dtype=torch.float32)
#     traj_src_np, traj_trans_np, _ = test_single_homeo_ds_net(homeo_ds_net=homeo_ds_net, trajectories_target=trajectories_target)

#     np.save(f"{save_dir_pertvf_sra}/traj_motif_transformed_{p_norm}.npy", traj_trans_np) 
#     np.save(f"{save_dir_pertvf_sra}/traj_motif_source_{p_norm}.npy", traj_src_np) #save trajectories_motif directly from the source 
